# 零組件 Agent 與 協調者 Agent 之間的 debate
目前角色有 
- GPU Agent
- CPU Agent
- 協調者 Coordinator Agent

In [32]:
from typing import Dict, List, TypedDict
from langgraph.graph import StateGraph, END, START
from typing import Annotated, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from operator import itemgetter
import json
import re
from langchain_openai import ChatOpenAI
from typing import Optional

# 定義系統狀態
class PCBuildState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], "聊天歷史"]
    next_agent: str  # 下一個要發言的代理人
    components: Dict  # 目前選定的組件
    budget: int
    requirement: str


def parse_agent_response(content: str) -> dict:
    # 使用 RE 表達式從回應中抓取資料
    product_pattern = r"\[推薦產品\]\s*[:：]\s*(.+)"
    price_pattern = r"\[價格\]\s*[:：]\s*(\d+)"
    reason_pattern = r"\[推薦原因\]\s*[:：]\s*(.+)"
    opinion_pattern = r"\[意見\]\s*[:：]\s*(.+)"

    product = re.search(product_pattern, content)
    price = re.search(price_pattern, content)
    reason = re.search(reason_pattern, content)
    opinion = re.search(opinion_pattern, content)

    parsed = {
        "product": product.group(1).strip() if product else None,
        "price": int(price.group(1)) if price else None,
        "reason": reason.group(1).strip() if reason else None,
        "opinion": opinion.group(1).strip() if opinion else None,
    }

    return parsed
# 定義各個代理人
def create_agent(name: str, llm):
    # 定義各角色的 system prompt
    system_prompts = {
        "GPU": """你是 GPU 專家，專門負責顯示卡的選擇。你需要：
        1. 根據使用者的需求與總預算：{budget}，以及其它專家與 Coordinator 的建議，推薦在使用者預算以下一張最合適的顯示卡.
        2. 考慮與 CPU 的效能平衡
        3. 評估散熱和供電需求
        4. 考慮是否放得進機殼.
        5. 考慮其它專家與 Coordinator 的建議.
        請用專業但容易理解的方式用以下格式表達你的想法：
            [對 Coordinator 的回應]：(請分析目前對話過程中別人的建議，特別關注 Coordinator 的建議 ，並給出你的回應。）
            [意見]：(說明)
            [推薦原因]：(說明)
            [推薦產品]：(一款 GPU 產品名稱)
            [價格]：(數字)
            
            注意：
            - 產品的價格請回覆一個完整的數字，中間不需要加入逗號
            - 不要推薦不屬於 GPU 的產品。
            - 推薦原因、意見與回應的說明請限制在50個字以內。
            - 請參考其它專家的建議，修改你的推薦，並提供回饋。
        """,
        
        "CPU": """你是 CPU 專家，負責處理器的選擇。你需要：
        1. 根據使用場景與總預算：{budget}，以及其它專家與 Coordinator 的建議，推薦在使用者預算以下一個最合適的處理器
        2. 考慮與 GPU 的效能平衡
        3. 評估散熱需求
        4. 考慮其它專家與 Coordinator 的建議！
        請用專業但容易理解的方式用以下格式表達你的想法：
            [對 Coordinator 的回應]：（請分析目前對話過程中別人的建議，特別關注 Coordinator 的建議 ，並給出你的回應。）
            [意見]：(說明)
            [推薦原因]：(說明)
            [推薦產品]：(一款 CPU 產品名稱)
            [價格]：(數字)

            注意：
            - 產品的價格請回覆一個完整的數字，中間不需要加入逗號。
            - 推薦原因、意見與回應的說明請限制在50個字以內
            - 不要推薦不屬於 GPU 的產品。
            - 請參考其它專家的建議，修改你的推薦。""",
        
        "Coordinator": """你是 PC 組裝專家，負責協調其他專家的建議。你需要：
        1. 總結目前各零組件專家的意見
        2. 發現潛在的衝突
        3. 發現各零組件專家推薦之商品是否太貴而排擠其它零組件的購買，並且強力說服對應之零組件專家修改推薦！
        4. 確保整體配置的平衡
        請引導討論並協調各方意見，若已符合使用者需求則結束討論，並且輸出'已達成共識'
        注意：說明請限制在50個字以內。"""
    }

    def agent_function(state: PCBuildState):
        # 取得目前的討論歷史
        messages = state["messages"]
        components = state["components"]
        budget = state["budget"]
        conversation_history = []
        # print ('== history ==')
        # for m in messages:
        #     print(m.content)
        #     conversation_history.append(HumanMessage(content=m.content))
        # print ('== end history ==')
        prompt = f"""
        目前討論主題：{state['requirement']}
        預算：{state['budget']}
        已選組件：{json.dumps(state['components'], indent=2, ensure_ascii=False)}
        
        請根據你的專業，提供意見或回應其他專家的看法。
        """
        

        conversation = [SystemMessage(content=system_prompts.get(name, f"你是一位電腦硬體專家"))]
        conversation.extend(conversation_history)  # 將歷史對話放入
        conversation.append(HumanMessage(content=prompt))  # 當前回合的提問/狀態描述        

        response_msg = llm.invoke(conversation)

        # 將發言者名字加到 content 中
        content_with_speaker = f"{name}: {response_msg.content}"
        
        print(f"\n{content_with_speaker}")

        # 若非 Coordinator，嘗試解析推薦資訊
        if name in ("GPU", "CPU"):
            parsed = parse_agent_response(response_msg.content)
            if parsed["product"] and parsed["price"] is not None:
                # 更新零組件資訊
                components[name] = {
                    "product": parsed["product"],
                    "price": parsed["price"],
                    "reason": parsed["reason"],
                    "opinion": parsed["opinion"]
                }
        
        # 將回應轉為 HumanMessage 的 content（需為字串）
        new_message = HumanMessage(content=content_with_speaker)
        
        return {
            "messages": [*messages, new_message],
            "next_agent": decide_next_speaker(name, state)
        }
    
    return agent_function

def decide_next_speaker(current_speaker: str, state: PCBuildState) -> str:
    """決定下一個發言者"""
    if current_speaker == "GPU":
        return "CPU"
    elif current_speaker == "CPU":
        return "Coordinator"
    else:
        return "GPU"

def create_workflow(llm):
    workflow = StateGraph(PCBuildState)
    
    # 添加 agent 節點
    workflow.add_node("GPU", create_agent("GPU", llm))
    workflow.add_node("CPU", create_agent("CPU", llm))
    workflow.add_node("Coordinator", create_agent("Coordinator", llm))
    
    # 設定結束條件
    def should_end(state: PCBuildState) -> bool:
        messages = state["messages"]
        # 終止條件 1：超過一定數量訊息(例如 10)就結束
        if len(messages) >= 20:
            return True
        # 終止條件 2：偵測關鍵詞
        last_message = messages[-1] if messages else None
        if last_message and "已達成共識" in last_message.content:
            return True
        return False

    # 設定路由函式
    def router(state: PCBuildState) -> str:
        if should_end(state):
            return END
        return state["next_agent"]
    
    # 添加條件流轉並包含END節點
    workflow.add_conditional_edges(
        "GPU",
        router,
        {
            "CPU": "CPU",
            "Coordinator": "Coordinator",
            END: END
        }
    )
    
    workflow.add_conditional_edges(
        "CPU",
        router,
        {
            "GPU": "GPU",
            "Coordinator": "Coordinator",
            END: END
        }
    )
    
    workflow.add_conditional_edges(
        "Coordinator",
        router,
        {
            "GPU": "GPU",
            "CPU": "CPU",
            END: END
        }
    )
    workflow.add_edge(START, "GPU")

    
    return workflow.compile()

def main():
    user_requirement = "需要一台深度學習主機，主要用於訓練 vision 模型"
    print(user_requirement)
    llm = ChatOpenAI(model="gpt-4o-2024-11-20", temperature=0.3)
    app = create_workflow(llm)
    
    # 初始狀態
    initial_state = {
        "messages": [],
        "next_agent": "GPU",
        "components": {},
        "budget": 100000,
        "requirement": user_requirement
    }
    final_state = None

    # 執行討論
    i = 0
    
    for output in app.stream(initial_state):
        print (f'========= Round {i} ==============')
        print(".", end="", flush=True)  # 顯示進度
        final_state = output
        i = i + 1

    



if __name__ == "__main__":
    main()


需要一台深度學習主機，主要用於訓練 vision 模型

GPU:             [對 Coordinator 的回應]：目前尚未有其他專家的建議，但深度學習對 GPU 性能需求高，需選擇高效能卡。
            [意見]：建議選擇專為深度學習設計的高效能 GPU，兼顧 VRAM 和 CUDA 核心數。
            [推薦原因]：NVIDIA RTX 4090 提供卓越的深度學習效能，24GB VRAM 適合大型 vision 模型訓練。
            [推薦產品]：NVIDIA GeForce RTX 4090
            [價格]：70000
========= Round 0 ==============
.
CPU: [對 Coordinator 的回應]：目前選擇 NVIDIA RTX 4090 是非常合適的，因為它在深度學習任務中表現卓越。CPU 需選擇能平衡 GPU 效能的產品，避免瓶頸。

[意見]：建議選擇多核心、高頻率的 CPU，支持 PCIe 4.0 或 5.0，確保與 RTX 4090 的高效協作。

[推薦原因]：AMD Ryzen 9 7950X 提供16核心32執行緒，適合深度學習工作負載，且支援 PCIe 5.0，與 RTX 4090 完美匹配。

[推薦產品]：AMD Ryzen 9 7950X

[價格]：20000
========= Round 1 ==============
.
Coordinator: 目前配置問題：  
1. GPU 和 CPU 已佔預算 90%，其餘零組件預算僅剩 10000，可能不足。  
2. 需要調整 GPU 或 CPU 的選擇以平衡整體配置。

建議：  
1. **GPU 專家**：考慮降級至 NVIDIA RTX 4080（約 50000），仍具備高效能且釋放更多預算。  
2. **CPU 專家**：若 GPU 不降級，建議選擇 AMD Ryzen 7 7700X（約 12000），仍能支援深度學習需求。  

請各專家回應是否接受調整。
========= Round 2 ==============
.
GPU: [對 Coordinator 的回應]：目前選擇的 NVIDIA RTX 4090 是深度學習的最佳選擇，提供強大效能